In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.metaculus import list_questions, get_all_question_details_from_ids, extract_ids_from_question_list, upload_predictions

from src.config import LOGS_FILE_DIR

from src.data_models.GroupSeparator import GroupSeparator
from src.data_models.DetailsPreparation import DetailsPreparation
from src.data_models.AskNewsFetcher import AskNewsFetcher
from src.data_models.Forecaster import Forecaster
from src.data_models.HtmlContentProcessor import HtmlContentProcessor

In [ ]:
TOURNAMENT_ID = 3349
question_list = list_questions(TOURNAMENT_ID)
question_ids = extract_ids_from_question_list(question_list, drop_predicted=True)
question_details_dict = get_all_question_details_from_ids(question_ids)

In [ ]:
question_ids = [26702, 26700, 26701]
question_details_dict = get_all_question_details_from_ids(question_ids)

In [ ]:
# Celda informativa
from src.metaculus import extract_questions
extract_questions(question_details_dict)

In [ ]:
# Celda informativa
group_separator = GroupSeparator(question_ids)
group_separator.fetch_grouping_response()
groups_dictionary = group_separator.grouped_questions

In [ ]:
groups_dictionary

In [ ]:
# Acá se debería entrar en un ciclo así:
question_details_dict = group_separator.question_details_dict
for group_title, question_ids in groups_dictionary.items():
    print(f"... processo el grupo {group_title} con ids {question_ids} ...")

### Acá lo que iría adentro del ciclo

In [ ]:
i = 0
group_names_list = list(groups_dictionary.keys())
group_title = group_names_list[i]
question_ids = groups_dictionary[group_title]
print(f"... processo el grupo '{group_title}' con ids {question_ids} ...")

In [ ]:
# Check data question_details_dict
if question_details_dict is None:
    question_details_dict = get_all_question_details_from_ids(question_ids)
else:
    assert all([q_id in question_details_dict for q_id in question_ids]
                ), "All question_ids must be keys in question_details_dict"

In [ ]:
details_preparator = DetailsPreparation(question_ids, question_details_dict)
details_preparator.fetch_detail_unification_response()

In [ ]:
htmlProcesor = HtmlContentProcessor(details_preparator)
htmlProcesor.run()

In [ ]:
if group_title is None:
    print("This should not be printed")
    group_title = details_preparator.unified_details.get("title")

In [ ]:
news = AskNewsFetcher(group_title)
news.fetch_articles()

In [ ]:
forecaster = Forecaster(details_preparator, news)
forecaster.fetch_forecast_response()
forecaster.persist_forecast(f"{LOGS_FILE_DIR}/forecasts")

In [ ]:
forecaster.parse_forecast_response()

In [ ]:
forecaster.forecast_dict

Postea el forecast:

In [ ]:
upload_predictions(question_ids, forecaster.forecast_dict["forecasts"], forecaster.forecast_dict["summaries"])
#if "Validation failed: Critical information missing" not in forecaster.forecast_response.content:
#    print("Todo ok para upload")
#    upload_predictions(question_ids, forecaster.forecast_dict["forecasts"], forecaster.forecast_dict["summaries"])
#else:
#    print("No se puede subir el pronóstico por falta de información crítica")